In [1]:
from UtilityLib import ProjectManager
T0104 = ProjectManager(path_base=r"C:\Users\Abhimanyu\OneDrive\Desktop\GenomicData\TCGA")


c:\Users\Abhimanyu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_sample_sheet = T0104.pd_tsv(T0104.find_files(T0104.get_path(), "gdc_manifest*")[0])
_sample_sheet.columns = [_col.replace(' ', '_') for _col in _sample_sheet.columns]
T0104.pd_excel(T0104.get_path("sample-data.xlsx"), _sample_sheet, "Samples")
_sample_sheet.head(2)

,id,filename,md5,size,state
0,2b03273b-6875-4e16-84a3-1c76c9000111,4b5d59e1-12d8-49b9-8b32-32401179ab0c.rna_seq.a...,2c74cf8d7b261bb2d74adc8725def338,4227241,released
1,c78c1b95-00af-4e5a-9c51-ac2b49cacaee,30d5bd02-e7d9-49f0-af23-6ae0986e0fe5.rna_seq.a...,73b71e7b5571da0cded5d15cb761a6cf,4227479,released


In [3]:
"""For star count"""
if not isinstance(T0104.config.exp.star_files, (list, tuple)):
    T0104.config.exp.star_files = []

def convert_file_set_to_df(T0104, _file_set, _chunk_path):
    _expression_df = None
    for _f in _file_set:
        _fn = T0104.filename(_f, with_ext=True)
        if _f in T0104.config.exp.star_files:
            continue
        _df = T0104.pd_tsv(_f, skiprows=1)
        _df['FileName'] = _fn
        _expression_df = _df if _expression_df is None else T0104.PD.concat([_expression_df, _df], axis=0)
        T0104.config.exp.star_files.append(_fn)
    _expression_df.to_pickle(_chunk_path) 


_expression_path = T0104.get_path('expression-star-count.pkl.gz')
if T0104.exists(_expression_path):
    _combined_df = T0104.unpickle(_expression_path)
else:
    print("Reprocessing Files...")
    T0104.validate_dir(T0104.get_path('Read-Chunks'))
    T0104.init_multiprocessing()
    _chunk_size = 20
    _expression_files = T0104.find_files(T0104.get_path("TCGA-Downloads"), "*/*tsv")
    _chunk_list = list(T0104.chunks(_expression_files, _chunk_size))
    for _chunk_idx, _file_set in T0104.PB(enumerate(_chunk_list), total=len(_chunk_list)):
        _chunk_path = T0104.get_path(f'Read-Chunks/File-Chunk-{_chunk_idx}.pkl.gz')
        if T0104.exists(_chunk_path):
            continue
        # convert_file_set_to_df(T0104, _file_set, _chunk_path)
        T0104.queue_task(convert_file_set_to_df, T0104, _file_set, _chunk_path)
        T0104.update_config()

    T0104.process_queue()
    """Combined all the read files"""
    _pkl_dfs = T0104.find_files(T0104.get_path("Read-Chunks"), "*pkl.gz")
    _combined_df = None
    for _pkf in T0104.PB(_pkl_dfs):
        _df = T0104.unpickle(_pkf)
        _combined_df = _df if _combined_df is None else T0104.PD.concat([_combined_df, _df], axis=0)
    _combined_df.to_pickle(_expression_path)
    print(_combined_df.shape) # ~ (96760675, 2)

print(_combined_df.head(2))

Reprocessing Files...


 50%|█████     | 40/80 [37:33<1:25:40, 128.51s/it]

In [19]:
_combined_df.head(20)

,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded,FileName
0,N_unmapped,NaN,NaN,1884156,1884156,1884156,NaN,NaN,NaN,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
1,N_multimapping,NaN,NaN,5772894,5772894,5772894,NaN,NaN,NaN,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
2,N_noFeature,NaN,NaN,3331800,41101584,41057602,NaN,NaN,NaN,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
3,N_ambiguous,NaN,NaN,6974507,1651293,1654961,NaN,NaN,NaN,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
4,ENSG00000000003.15,TSPAN6,protein_coding,4370,2153,2217,56.2216,13.5542,13.0317,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
5,ENSG00000000005.6,TNMD,protein_coding,7,2,5,0.2768,0.0667,0.0642,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
6,ENSG00000000419.13,DPM1,protein_coding,2625,1314,1312,126.9161,30.5977,29.4181,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
7,ENSG00000000457.14,SCYL3,protein_coding,3005,2389,2252,25.4778,6.1423,5.9055,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
8,ENSG00000000460.17,C1orf112,protein_coding,1578,1728,1697,15.4251,3.7188,3.5754,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...
9,ENSG00000000938.13,FGR,protein_coding,599,301,298,10.3359,2.4918,2.3958,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...


In [ ]:
T0104.queue_task_status()

In [ ]:
_combined_df.shape

In [ ]:
_combined_df.head(2)

In [ ]:
_filter_exp = _combined_df.gene_id.str.startswith('N_')

_mapping_data = _combined_df[_filter_exp].copy()
_expression_data = _combined_df[~_filter_exp].copy()

_mapping_data.shape, _expression_data.shape